In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import shap

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
os.chdir('/Users/sunnyyou/Programs/Real_Time_HAI/HIECNN/IMERG')

In [3]:
train = pd.read_csv("DEV/ALL_TRAIN_DATA_RESAMPLE.csv")
train = train[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12"]]
print(len(train.GIS_ID))
train.head()

58112


,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_200001_TD_2000060806,25,25,25
1,ATL_200001_TD_2000060812,25,25,25
2,ATL_200002_TD_2000062312,30,30,25
3,ATL_200002_TD_2000062318,30,30,30
4,ATL_200002_TD_2000062400,30,30,30


In [4]:
test = pd.read_csv("DEV/ALL_TEST_DATA.csv")
test = test[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12"]]
print(len(test.GIS_ID))
test.head()

4312


,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_201801_TS_2018052812,50,55,55
1,ATL_201801_TS_2018052818,45,50,55
2,ATL_201801_TD_2018052900,30,45,50
3,ATL_201801_TD_2018052906,30,30,45
4,ATL_201801_TD_2018052912,30,30,30


In [5]:
a = pd.read_csv("IMERG_CSV/" + train.GIS_ID[0] + '.csv', header = None)
print(a.head())
pvmax = np.array([train.VMAX_N06[0], train.VMAX_N12[0]])
print(pvmax)

        0         1         2         3         4         5    6         7    \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.000000   
2  0.000000  0.000000  0.002614  0.035389  0.031253  0.003356  0.0  0.000000   
3  0.053374  0.086443  0.075739  0.183154  0.187500  0.022317  0.0  0.046795   
4  0.312158  0.434224  0.340077  0.413843  0.241398  0.028628  0.0  0.202615   

        8         9    ...       111       112       113  114  115  116  117  \
0  0.000000  0.000000  ...  0.327364  0.000000  0.000000  0.0  0.0  0.0  0.0   
1  0.000000  0.000000  ...  0.000000  0.000000  0.022034  0.0  0.0  0.0  0.0   
2  0.000000  0.000000  ...  0.000000  0.000000  0.022034  0.0  0.0  0.0  0.0   
3  0.354221  0.840079  ...  0.000000  0.295887  0.192012  0.0  0.0  0.0  0.0   
4  1.544581  4.629189  ...  0.100727  0.147943  0.047216  0.0  0.0  0.0  0.0   

   118  119  120  
0  0.0  0.0  0.0  


In [6]:
train_img = []
train_vmax = []
train_label = []
test_img = []
test_vmax = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = f"IMERG_CSV/{train.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.VMAX[f]
        train_label.append(lab)
        pvmax = np.array([train.VMAX_N06[f], train.VMAX_N12[f]])
        train_vmax.append(pvmax)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = f"IMERG_CSV/{test.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.VMAX[f]
        test_label.append(lab)
        pvmax = np.array([test.VMAX_N06[f], test.VMAX_N12[f]])
        test_vmax.append(pvmax)
    except Exception as e:
        pass

In [7]:
print(len(train_img))
print(len(train_vmax))
print(len(train_label))
print(len(test_img))
print(len(test_vmax))
print(len(test_label))

58112
58112
58112
4312
4312
4312


In [8]:
X_train_img = train_img
X_train_vmax = train_vmax
y_train = train_label
X_test_img = test_img
X_test_vmax = test_vmax
y_test = test_label

In [9]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,41,41,1)
X_train_img = X_train_img.astype('float32')
X_train_vmax = np.array(X_train_vmax)
X_train_vmax = X_train_vmax.reshape(-1,2)
y_train = np.array(y_train)

In [10]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,41,41,1)
X_test_img = X_test_img.astype('float32')
X_test_vmax = np.array(X_test_vmax)
X_test_vmax = X_test_vmax.reshape(-1,2)
y_test = np.array(y_test)

In [11]:
# 300km
vmax_input = keras.Input(shape =(2,), name = "vmax_layer")
img_input = keras.Input(shape =(61, 61, 1), name = "img_layer")

w = keras.layers.Conv2D(64,12) (img_input)
w = keras.layers.Conv2D(64,12)(w)
w = keras.layers.Conv2D(64,2)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.linear(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(256,2)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, vmax_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(170, activation = 'linear')(output_layer1)

new_model1 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 12) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,2, activation = 'linear')(x)
x = keras.layers.Conv2D(128,7)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,2, activation = 'linear')(x)
x = keras.layers.Conv2D(64,4)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, vmax_input])
output_layer2 = keras.layers.Dense(170, activation = 'linear')(merged_model2)

new_model2 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (10,4))(img_input)
y = keras.layers.Conv2D(256, (4,10))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,6)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, vmax_input])
output_layer3 = keras.layers.Dense(170, activation = 'linear')(merged_model3)

new_model3 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

# z = keras.layers.Conv2D(256, 10)(img_input)
# z = keras.layers.MaxPool2D(2,2)(z)
# img_output4 = keras.layers.Flatten()(z)

# merged_model4 = keras.layers.concatenate([img_output4, vmax_input])
# output_layer4 = keras.layers.Dense(1, activation = 'sigmoid')(merged_model4)

# new_model4 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer4, name = "model_4")

# new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_layer (InputLayer)      [(None, 61, 61, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 50, 50, 64)           9280      ['img_layer[0][0]']           
                                                                                                  
 conv2d_1 (Conv2D)           (None, 39, 39, 64)           589888    ['conv2d[0][0]']              


2024-07-17 19:22:18.397456: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-07-17 19:22:18.397482: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-07-17 19:22:18.397491: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-07-17 19:22:18.397521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-17 19:22:18.397534: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


                                                                                                  
 conv2d_2 (Conv2D)           (None, 38, 38, 64)           16448     ['conv2d_1[0][0]']            
                                                                                                  
 batch_normalization (Batch  (None, 38, 38, 64)           256       ['conv2d_2[0][0]']            
 Normalization)                                                                                   
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 19, 19, 64)           0         ['batch_normalization[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv2d_3 (Conv2D)           (None, 11, 11, 64)           331840    ['max_pooling2d[0][0]']       
          

In [12]:
# 200km
vmax_input = keras.Input(shape =(2,), name = "vmax_layer")
img_input = keras.Input(shape =(41, 41, 1), name = "img_layer")

w = keras.layers.Conv2D(64,8) (img_input)
w = keras.layers.Conv2D(64,8)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, vmax_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(170)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 8) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,3)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, vmax_input])
output_layer2 = keras.layers.Dense(170)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (6,2))(img_input)
y = keras.layers.Conv2D(256, (2,6))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, vmax_input])
output_layer3 = keras.layers.Dense(170)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_layer (InputLayer)      [(None, 41, 41, 1)]          0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 34, 34, 64)           4160      ['img_layer[0][0]']           
                                                                                                  
 conv2d_16 (Conv2D)          (None, 27, 27, 64)           262208    ['conv2d_15[0][0]']           
                                                                                                  
 conv2d_17 (Conv2D)          (None, 27, 27, 64)           4160      ['conv2d_16[0][0]']           
                                                                                            

In [ ]:
os.chdir("../")
models = {new_model1: '1', new_model2: '2', new_model3: '3'}
ep = range(10, 11)
batches = [1, 2, 4, 8, 16]
for m in models.keys():
    for e in ep:
        for b in batches:
            with tf.device('/GPU:0'):
                m.compile(optimizer = 'adam', loss = tf.keras.losses.MeanAbsoluteError(), metrics = ['mae', 'mse'])
                m.fit([X_train_img, X_train_vmax], y_train, epochs = e, batch_size = b)
                tf.keras.models.save_model(m, f"MODELS/MODEL{models[m]}_EPOCHS{e}_BATCH{b}")
                res = m.evaluate([X_test_img, X_test_vmax], y_test)
                print("MAE = " + str(res[0]))
                print("RMSE = " + str((res[2]) ** 0.5))
                preds = m.predict([X_test_img, X_test_vmax])
                x = np.average(preds, axis = 1)
                test['preds'] = x
                test.to_csv(f"OUTPUT/MODEL{models[m]}_RESULTS{e}_{b}.csv")
                tf.keras.backend.clear_session()

Epoch 1/10
58112/58112 [==============================] - 461s 8ms/step - loss: 5.5928 - mae: 5.5928 - mse: 79.5129
Epoch 2/10
58112/58112 [==============================] - 460s 8ms/step - loss: 4.5713 - mae: 4.5713 - mse: 43.7893
Epoch 3/10
58112/58112 [==============================] - 467s 8ms/step - loss: 4.5177 - mae: 4.5177 - mse: 43.2209
Epoch 4/10
58112/58112 [==============================] - 470s 8ms/step - loss: 4.4877 - mae: 4.4877 - mse: 42.6941
Epoch 5/10
58112/58112 [==============================] - 474s 8ms/step - loss: 4.4699 - mae: 4.4699 - mse: 42.1806
Epoch 6/10
58112/58112 [==============================] - 474s 8ms/step - loss: 4.4522 - mae: 4.4522 - mse: 42.1709
Epoch 7/10
58112/58112 [==============================] - 474s 8ms/step - loss: 4.4463 - mae: 4.4463 - mse: 42.0436
Epoch 8/10
58112/58112 [==============================] - 475s 8ms/step - loss: 4.4352 - mae: 4.4352 - mse: 41.8772
Epoch 9/10
58112/58112 [==============================] - 475s 8ms/step 

INFO:tensorflow:Assets written to: MODELS/MODEL1_EPOCHS10_BATCH1/assets


135/135 [==============================] - 2s 11ms/step - loss: 3.9599 - mae: 3.9599 - mse: 29.9316
MAE = 3.959933280944824
RMSE = 5.470979247092712
135/135 [==============================] - 1s 7ms/step
Epoch 1/10
29056/29056 [==============================] - 260s 9ms/step - loss: 4.3100 - mae: 4.3100 - mse: 40.3517
Epoch 2/10
29056/29056 [==============================] - 259s 9ms/step - loss: 4.2977 - mae: 4.2977 - mse: 40.0431
Epoch 3/10
29056/29056 [==============================] - 259s 9ms/step - loss: 4.2901 - mae: 4.2901 - mse: 39.8170
Epoch 4/10
29056/29056 [==============================] - 259s 9ms/step - loss: 4.2915 - mae: 4.2915 - mse: 39.8423
Epoch 5/10
29056/29056 [==============================] - 258s 9ms/step - loss: 4.2857 - mae: 4.2857 - mse: 39.7654
Epoch 6/10
29056/29056 [==============================] - 256s 9ms/step - loss: 4.2812 - mae: 4.2812 - mse: 39.6229
Epoch 7/10
29056/29056 [==============================] - 256s 9ms/step - loss: 4.2711 - mae: 4.2711

INFO:tensorflow:Assets written to: MODELS/MODEL1_EPOCHS10_BATCH2/assets


135/135 [==============================] - 1s 9ms/step - loss: 3.8620 - mae: 3.8620 - mse: 28.1146
MAE = 3.8619744777679443
RMSE = 5.302318375429652
135/135 [==============================] - 1s 6ms/step
Epoch 1/10
14528/14528 [==============================] - 143s 10ms/step - loss: 4.1710 - mae: 4.1710 - mse: 38.1621
Epoch 2/10
14528/14528 [==============================] - 142s 10ms/step - loss: 4.1644 - mae: 4.1644 - mse: 38.0175
Epoch 3/10
14528/14528 [==============================] - 142s 10ms/step - loss: 4.1611 - mae: 4.1611 - mse: 37.8714
Epoch 4/10
14528/14528 [==============================] - 142s 10ms/step - loss: 4.1582 - mae: 4.1582 - mse: 37.8866
Epoch 5/10
14528/14528 [==============================] - 141s 10ms/step - loss: 4.1590 - mae: 4.1590 - mse: 37.7279
Epoch 6/10
14528/14528 [==============================] - 141s 10ms/step - loss: 4.1390 - mae: 4.1390 - mse: 37.4763
Epoch 7/10
14528/14528 [==============================] - 141s 10ms/step - loss: 4.1510 - mae:

...

135/135 [==============================] - 2s 12ms/step - loss: 3.8087 - mae: 3.8087 - mse: 27.0706
MAE = 3.808690309524536
RMSE = 5.202940058356861
135/135 [==============================] - 2s 11ms/step
Epoch 1/10
7264/7264 [==============================] - 90s 12ms/step - loss: 2.1949 - mae: 2.1949 - mse: 13.2629
Epoch 2/10
7264/7264 [==============================] - 90s 12ms/step - loss: 2.1387 - mae: 2.1387 - mse: 12.7762
Epoch 3/10
7264/7264 [==============================] - 95s 13ms/step - loss: 2.0881 - mae: 2.0881 - mse: 12.3806
Epoch 4/10
7264/7264 [==============================] - 101s 14ms/step - loss: 2.0683 - mae: 2.0683 - mse: 12.1410
Epoch 5/10
7264/7264 [==============================] - 106s 15ms/step - loss: 2.0436 - mae: 2.0436 - mse: 11.8237
Epoch 6/10
7264/7264 [==============================] - 107s 15ms/step - loss: 2.0146 - mae: 2.0146 - mse: 11.5988
Epoch 7/10
7264/7264 [==============================] - 106s 15ms/step - loss: 1.9936 - mae: 1.9936 - mse: 1

INFO:tensorflow:Assets written to: MODELS/MODEL3_EPOCHS10_BATCH8/assets


135/135 [==============================] - 2s 14ms/step - loss: 3.9242 - mae: 3.9242 - mse: 28.7158
MAE = 3.9242281913757324
RMSE = 5.358708020404659
135/135 [==============================] - 2s 14ms/step
Epoch 1/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.7710 - mae: 1.7710 - mse: 9.9223
Epoch 2/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.7242 - mae: 1.7242 - mse: 9.6106
Epoch 3/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.7009 - mae: 1.7009 - mse: 9.3892
Epoch 4/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.6833 - mae: 1.6833 - mse: 9.3157
Epoch 5/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.6626 - mae: 1.6626 - mse: 9.0812
Epoch 6/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.6496 - mae: 1.6496 - mse: 9.0508
Epoch 7/10
3632/3632 [==============================] - 87s 24ms/step - loss: 1.6373 - mae: 1.6373 - mse: 8.8658
Epo

INFO:tensorflow:Assets written to: MODELS/MODEL3_EPOCHS10_BATCH16/assets


135/135 [==============================] - 2s 14ms/step - loss: 3.9659 - mae: 3.9659 - mse: 29.3017
MAE = 3.96587872505188
RMSE = 5.413103114965755
135/135 [==============================] - 2s 14ms/step


In [13]:
os.chdir("/Users/sunnyyou/Programs/Real_Time_HAI/HIECNN/IMERG")

In [14]:
train_no_resample = pd.read_csv("DEV/ALL_TRAIN_DATA.csv")
train_no_resample = train_no_resample[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12", "CAT"]]
train_no_resample.head()

,GIS_ID,VMAX,VMAX_N06,VMAX_N12,CAT
0,ATL_200001_TD_2000060806,25,25,25,TD
1,ATL_200001_TD_2000060812,25,25,25,TD
2,ATL_200002_TD_2000062312,30,30,25,TD
3,ATL_200002_TD_2000062318,30,30,30,TD
4,ATL_200002_TD_2000062400,30,30,30,TD


In [15]:
test = pd.read_csv("DEV/ALL_TEST_DATA.csv")
test = test[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12", "CAT"]]
test.head()

,GIS_ID,VMAX,VMAX_N06,VMAX_N12,CAT
0,ATL_201801_TS_2018052812,50,55,55,TS
1,ATL_201801_TS_2018052818,45,50,55,TS
2,ATL_201801_TD_2018052900,30,45,50,TD
3,ATL_201801_TD_2018052906,30,30,45,TD
4,ATL_201801_TD_2018052912,30,30,30,TD


In [16]:
train_no_resample_TD = train_no_resample[train_no_resample['CAT'] == 'TD']
train_no_resample_TD = train_no_resample_TD.sample(100)
train_no_resample_TS = train_no_resample[train_no_resample['CAT'] == 'TS']
train_no_resample_TS = train_no_resample_TS.sample(100)
train_no_resample_Min = train_no_resample[train_no_resample['CAT'] == 'Min']
train_no_resample_Min = train_no_resample_Min.sample(100)
train_no_resample_Maj = train_no_resample[train_no_resample['CAT'] == 'Maj']
train_no_resample_Maj = train_no_resample_Maj.sample(100)
shap_train = pd.concat([train_no_resample_TD, train_no_resample_TS, train_no_resample_Min, train_no_resample_Maj])
shap_train = shap_train.reset_index(drop = True)
shap_train
test_TD = test[test['CAT'] == 'TD']
test_TS = test[test['CAT'] == 'TS']
test_Min = test[test['CAT'] == 'Min']
test_Maj = test[test['CAT'] == 'Maj']
test_TD = test_TD.sample(100)
test_TS = test_TS.sample(100)
test_Min = test_Min.sample(100)
test_Maj = test_Maj.sample(100)
shap_test = pd.concat([test_TD, test_TS, test_Min, test_Maj])
shap_test = shap_test.reset_index(drop = True)
shap_test

,GIS_ID,VMAX,VMAX_N06,VMAX_N12,CAT
0,NIO_201906_TD_2019120400,30,30,30,TD
1,NWP_201833_TD_2018112600,25,30,35,TD
2,NWP_201929_TD_2019120518,25,30,30,TD
3,NIO_201904_TD_2019110206,25,25,25,TD
4,SPA_201909_TD_2019010200,30,25,25,TD
...,...,...,...,...,...
395,CPA_201801_C5_2018100200,140,130,110,Maj
396,EPA_201810_C4_2018080606,125,120,115,Maj
397,SIO_201807_C3_2018020106,105,105,110,Maj
398,CPA_201906_C4_2019073100,115,115,100,Maj


In [17]:
shap_train_img = []
shap_train_ships = []
shap_train_label = []
shap_test_img = []
shap_test_ships = []
shap_test_label = []
for f in range(len(shap_train.GIS_ID)) :
    filename = f"IMERG_CSV/{shap_train.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        shap_train_img.append(temp)
        lab = shap_train.VMAX[f]
        shap_train_label.append(lab)
        pvmax = np.array([shap_train.VMAX_N06[f], shap_train.VMAX_N12[f]])
        shap_train_ships.append(pvmax)
    except Exception as e:
        pass
for f in range(len(shap_test.GIS_ID)):
    filename = f"IMERG_CSV/{shap_test.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        shap_test_img.append(temp)
        lab = shap_test.VMAX[f]
        shap_test_label.append(lab)
        pvmax = np.array([shap_test.VMAX_N06[f], shap_test.VMAX_N12[f]])
        shap_test_ships.append(pvmax)
    except Exception as e:
        pass

In [18]:
shap_train_img = np.array(shap_train_img)
shap_train_img = shap_train_img.reshape(-1,41,41,1)
shap_train_img = shap_train_img.astype('float32')
shap_train_ships = np.array(shap_train_ships)
shap_train_ships = shap_train_ships.reshape(-1,2)
shap_train_label = np.array(shap_train_label)

In [19]:
shap_test_img = np.array(shap_test_img)
shap_test_img = shap_test_img.reshape(-1,41,41,1)
shap_test_img = shap_test_img.astype('float32')
shap_test_ships = np.array(shap_test_ships)
shap_test_ships = shap_test_ships.reshape(-1,2)
shap_test_label = np.array(shap_test_label)

In [20]:
len(shap_test_img)

400

In [21]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [22]:
new_model2.fit([X_train_img, X_train_vmax], y_train, epochs=1, batch_size=4, validation_split = 0.1)
res = new_model2.evaluate([X_test_img, X_test_vmax], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))
preds = new_model2.predict([X_test_img, X_test_vmax])
a = np.average(preds, axis = 1)
test['preds'] = a

2024-07-17 19:22:21.719786: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


135/135 [==============================] - 1s 9ms/step - loss: 4.2242 - mae: 4.2242 - mse: 34.7757
MAE = 4.224246501922607
RMSE = 5.897095058845214
135/135 [==============================] - 1s 6ms/step


In [23]:
shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.linearity_1d(0)
e = shap.DeepExplainer(new_model2, [shap_train_img, shap_train_ships])
shap_values = e.shap_values([shap_test_img, shap_test_ships])

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [56]:
np.shape(shap_values[0][1][0])

(2,)

In [57]:
shap_images = []
for node in range(170):
    for images in range(400):
        shap_images.append(np.sum(shap_values[node][0][images]))

shap_pvmax = []
for node in range(170):
    for images in range(400):
        for ships in range(2):
            shap_pvmax.append(np.sum(shap_values[node][1][images][ships]))

In [58]:
shap_pvmax

[-34.77124224677682,
 2.3610073909163476,
 -34.77124224677682,
 2.0198213517665864,
 -34.77124224677682,
 2.3610073909163476,
 -39.65655903592706,
 2.702193430066109,
 -39.65655903592706,
 2.702193430066109,
 -34.77124224677682,
 2.3610073909163476,
 -29.88592545762658,
 1.337449273467064,
 -34.77124224677682,
 2.0198213517665864,
 -34.77124224677682,
 2.3610073909163476,
 -39.65655903592706,
 2.702193430066109,
 -39.65655903592706,
 2.702193430066109,
 -39.65655903592706,
 2.702193430066109,
 -39.65655903592706,
 2.702193430066109,
 -39.65655903592706,
 2.3610073909163476,
 -39.65655903592706,
 2.3610073909163476,
 -34.77124224677682,
 2.3610073909163476,
 -34.77124224677682,
 2.3610073909163476,
 -39.65655903592706,
 2.3610073909163476,
 -29.88592545762658,
 1.6786353126168252,
 -39.65655903592706,
 2.702193430066109,
 -34.77124224677682,
 2.0198213517665864,
 -39.65655903592706,
 2.702193430066109,
 -34.77124224677682,
 2.0198213517665864,
 -34.77124224677682,
 2.3610073909163476,
 

In [59]:
shap_pvmax6 = [shap_pvmax[i] for i in range(0, len(shap_pvmax), 2)]
shap_pvmax12 = [shap_pvmax[i] for i in range(1, len(shap_pvmax), 2)]

In [63]:
for shaps in [shap_pvmax6, shap_pvmax12]:
    print(np.max(shaps))

91.49839530140162
4.399772554636002


In [64]:
shap_images

[-2.3467407831703184,
 -4.060980042821162,
 -3.282517559966447,
 -2.409876937127571,
 -1.6910069077134544,
 -2.9888315028829346,
 -3.205330864838564,
 -1.2241113783088124,
 -2.7228537511329876,
 -2.8947799596316592,
 -0.6642325290022866,
 0.24258513775386348,
 -1.9082778626950763,
 -2.2111262955107414,
 -3.008943780964703,
 -3.399134010096531,
 -3.4036515423421747,
 -3.1639032804449876,
 -3.0980780659163862,
 -2.9912425761279042,
 -3.2257130598758366,
 -2.6109871631981543,
 -3.327400857370625,
 -2.8241660579380543,
 -2.581614900498222,
 -3.2924963486427288,
 -3.3231990018689075,
 -2.839971310481804,
 -3.1680836601951334,
 -2.6365845357773594,
 -3.144488047685254,
 -3.370477374921336,
 -2.632963647157858,
 -2.4955293747167593,
 -3.3801540149570606,
 -2.445407544883496,
 -3.199913924049966,
 -2.0005377336040944,
 -2.8313158222821073,
 -1.137910195017728,
 -3.8172735105765483,
 -3.4272339726382093,
 -3.08040090979042,
 -0.5576091900032338,
 -3.138420795257483,
 -2.8985312207974268,
 -3.35

In [68]:
np.max(shap_images)

11.768011835516731